<a href="https://colab.research.google.com/github/Mhmmd-Mhmmd/Quera_questions/blob/main/06_transfer_learning_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import zipfile

In [ ]:
!wget https://storage.googleapis.com./ztm_tf_course/food_vision/101_food_classes_10_percent.zip
zip_ref = zipfile.ZipFile(file='./101_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()
!rm 101_food_classes_10_percent.zip

--2023-10-08 07:34:23--  https://storage.googleapis.com./ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com. (storage.googleapis.com.)... 142.251.2.207, 74.125.137.207, 2607:f8b0:4023:c0d::cf, ...
Connecting to storage.googleapis.com. (storage.googleapis.com.)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G   109MB/s    in 14s     

2023-10-08 07:34:37 (110 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory='./101_food_classes_10_percent/train/',
                                                                 batch_size=32,
                                                                 image_size=(224, 224),
                                                                 shuffle=True,
                                                                 label_mode='categorical')

valid_data = tf.keras.preprocessing.image_dataset_from_directory(directory='./101_food_classes_10_percent/test/',
                                                                 batch_size=32,
                                                                 label_mode='categorical',
                                                                 shuffle=False,
                                                                 image_size=(224, 224))

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [ ]:
data_augmentation = tf.keras.Sequential(layers=[tf.keras.layers.experimental.preprocessing.RandomFlip(),
                                                tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
                                                tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
                                                tf.keras.layers.experimental.preprocessing.RandomHeight(0.2),
                                                tf.keras.layers.experimental.preprocessing.RandomWidth(0.2),
                                                tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),], # & rescaling layer for cases like resnet
                                        name='augmentation_layer')

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224, 224, 3), name='input_layer')

x = data_augmentation(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name='vectorizing_layer')(x)

outputs = tf.keras.layers.Dense(units=len(train_data.class_names), activation=tf.keras.activations.softmax, name='classifier')(x)

model_0 = tf.keras.Model(inputs, outputs)

model_0.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics='accuracy')

model_0_history_featute = model_0.fit(
    train_data,
    validation_data=valid_data,
    steps_per_epoch=len(train_data),
    validation_steps=int(0.25 * len(valid_data)),
    epochs=5,
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/model_0/featue/',
                                              write_images=True),
                tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints/model_0/featue/model_0_feature.ckpt',
                                                    save_best_only=True,
                                                    save_weights_only=True)])

16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/5
237/237 [==============================] - 129s 480ms/step - loss: 3.4905 - accuracy: 0.2519 - val_loss: 2.6289 - val_accuracy: 0.4132
Epoch 2/5
237/237 [==============================] - 88s 371ms/step - loss: 2.3722 - accuracy: 0.4549 - val_loss: 2.1374 - val_accuracy: 0.4865
Epoch 3/5
237/237 [==============================] - 79s 331ms/step - loss: 2.0066 - accuracy: 0.5267 - val_loss: 1.9664 - val_accuracy: 0.5049
Epoch 4/5
237/237 [==============================] - 68s 287ms/step - loss: 1.7926 - accuracy: 0.5632 - val_loss: 1.8633 - val_accuracy: 0.5260
Epoch 5/5
237/237 [==============================] - 69s 288ms/step - loss: 1.6423 - accuracy: 0.5914 - val_loss: 1.8246 - val_accuracy: 0.5252


In [ ]:
result_feature = model_0.evaluate(valid_data)
result_feature

790/790 [==============================] - 58s 74ms/step - loss: 1.7093 - accuracy: 0.5522


[1.7092682123184204, 0.5521584153175354]

In [ ]:
for layer in base_model.layers[-10:]:
    layer.trainable = True

model_0.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), # 10X lesses
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics='accuracy')

model_0_history_fine = model_0.fit(
    train_data,
    validation_data=valid_data,
    steps_per_epoch=len(train_data),
    validation_steps=int(0.25 * len(valid_data)),
    epochs=10,
    initial_epoch=model_0_history_featute.epoch[-1],
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/model_0/fine/',
                                              write_images=True),
                tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints/model_0/fine/model_0_fine.ckpt',
                                                    save_best_only=True,
                                                    save_weights_only=True)])

Epoch 5/10
237/237 [==============================] - 73s 279ms/step - loss: 1.4782 - accuracy: 0.6465 - val_loss: 1.8152 - val_accuracy: 0.5295
Epoch 6/10
237/237 [==============================] - 59s 246ms/step - loss: 1.4439 - accuracy: 0.6511 - val_loss: 1.8085 - val_accuracy: 0.5317
Epoch 7/10
237/237 [==============================] - 57s 236ms/step - loss: 1.4391 - accuracy: 0.6540 - val_loss: 1.8052 - val_accuracy: 0.5343
Epoch 8/10
237/237 [==============================] - 55s 232ms/step - loss: 1.4341 - accuracy: 0.6552 - val_loss: 1.8033 - val_accuracy: 0.5332
Epoch 9/10
237/237 [==============================] - 58s 246ms/step - loss: 1.4131 - accuracy: 0.6586 - val_loss: 1.8005 - val_accuracy: 0.5338
Epoch 10/10
237/237 [==============================] - 57s 238ms/step - loss: 1.4049 - accuracy: 0.6619 - val_loss: 1.7923 - val_accuracy: 0.5336


In [ ]:
result_fine = model_0.evaluate(valid_data)
result_fine

790/790 [==============================] - 60s 76ms/step - loss: 1.6576 - accuracy: 0.5641


[1.6575735807418823, 0.5641188025474548]

In [ ]:
base_model = tf.keras.applications.EfficientNetB7(include_top=False)
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224, 224, 3), name='input_layer')

x = data_augmentation(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name='vectorizer')(x)

outputs = tf.keras.layers.Dense(units=len(train_data.class_names), activation='softmax', name='classifier')(x)

model_1 = tf.keras.Model(inputs, outputs)

model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics='accuracy')

model_1_history_feature = model_1.fit(
    train_data,
    steps_per_epoch=len(train_data),
    validation_data=valid_data,
    validation_steps=int(0.25*len(valid_data)),
    epochs=5,
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/model_1_feature/',
                                              write_graph=True,
                                              write_images=True),
               tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints/model_1_feature.ckpt',
                                                  save_weights_only=True,
                                                  save_best_only=True)])

Epoch 1/5
237/237 [==============================] - 233s 892ms/step - loss: 3.4402 - accuracy: 0.2399 - val_loss: 2.5932 - val_accuracy: 0.3834
Epoch 2/5
237/237 [==============================] - 210s 886ms/step - loss: 2.4511 - accuracy: 0.4169 - val_loss: 2.1916 - val_accuracy: 0.4535
Epoch 3/5
237/237 [==============================] - 207s 874ms/step - loss: 2.1315 - accuracy: 0.4821 - val_loss: 2.0694 - val_accuracy: 0.4743
Epoch 4/5
237/237 [==============================] - 189s 798ms/step - loss: 1.9610 - accuracy: 0.5141 - val_loss: 1.9999 - val_accuracy: 0.4894
Epoch 5/5
237/237 [==============================] - 201s 846ms/step - loss: 1.8171 - accuracy: 0.5450 - val_loss: 1.9210 - val_accuracy: 0.5005


In [ ]:
results_model_1_feature = model_1.evaluate(valid_data)
results_model_1_feature

790/790 [==============================] - 294s 372ms/step - loss: 1.7830 - accuracy: 0.5409


[1.7829984426498413, 0.5409108996391296]

In [ ]:
for layer in base_model.layers[-10:]:
    layer.trainable = True

model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics='accuracy')

model_1_history_fine = model_1.fit(
    train_data,
    steps_per_epoch=len(train_data),
    validation_data=valid_data,
    validation_steps=int(0.25*len(valid_data)),
    epochs=10,
    initial_epoch=model_1_history_feature.epoch[-1],
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir='logs/model_1_fine/',
                                              write_graph=True,
                                              write_images=True),
               tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints/model_1_fine.ckpt',
                                                  save_weights_only=True,
                                                  save_best_only=True)])

Epoch 5/10
237/237 [==============================] - 226s 862ms/step - loss: 1.6603 - accuracy: 0.5876 - val_loss: 1.9132 - val_accuracy: 0.5087
Epoch 6/10
237/237 [==============================] - 186s 783ms/step - loss: 1.6139 - accuracy: 0.6000 - val_loss: 1.9085 - val_accuracy: 0.5098
Epoch 7/10
237/237 [==============================] - 185s 780ms/step - loss: 1.6078 - accuracy: 0.6028 - val_loss: 1.8980 - val_accuracy: 0.5127
Epoch 8/10
237/237 [==============================] - 197s 828ms/step - loss: 1.5992 - accuracy: 0.6078 - val_loss: 1.8963 - val_accuracy: 0.5108
Epoch 9/10
237/237 [==============================] - 185s 776ms/step - loss: 1.5890 - accuracy: 0.6032 - val_loss: 1.8840 - val_accuracy: 0.5165
Epoch 10/10
237/237 [==============================] - 190s 801ms/step - loss: 1.5774 - accuracy: 0.6137 - val_loss: 1.8807 - val_accuracy: 0.5174


In [ ]:
results_model_1_fine = model_1.evaluate(valid_data)
results_model_1_fine

790/790 [==============================] - 293s 371ms/step - loss: 1.7307 - accuracy: 0.5531


[1.7306791543960571, 0.5530692934989929]

In [ ]:
def load_prep_images(filename: srt, image_shape=(224, 224), scale=True):
    img = tf.io.read_file(filename)
    img = tg.io.decode_image(image, channels=30)
    img = tf.image.resize(img, imahe_shape)

    if scale:
        return img/225.
    else:
        return img